In [1]:
import spice
import polars as pl
import altair as alt

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
# https://dune.com/queries/3524918/5927812
blob_market_fees_df = spice.query(3524918).with_columns(
    pl.col("hr")
    .str.strptime(pl.Date, "%Y-%m-%d %H:%M:%S%.f %Z", strict=False)
    .dt.truncate("1d").alias('day')
)

loading result from cache


In [3]:
blob_market_fees_df.schema

Schema([('hr', String),
        ('sum_blob_cost', Float64),
        ('sum_blob_usd', Float64),
        ('cumulative_execution_base_usd', Float64),
        ('cumulative_execution_priority_usd', Float64),
        ('cumulative_execution_base_eth', Float64),
        ('cumulative_execution_priority_eth', Float64),
        ('cumulative_consensus_usd', Float64),
        ('cumulative_consensus_eth', Float64),
        ('day', Date)])

In [4]:
# Reverse the cumulative values to get non-cumulative values
non_cumulative_blob_fees_df = blob_market_fees_df.select([
    "day",  # Keep the timestamp column
    pl.col("sum_blob_cost"),
    pl.col("sum_blob_usd"),
    pl.col("cumulative_execution_base_usd").diff().fill_null(
        strategy="zero").alias("execution_base_usd"),
    pl.col("cumulative_execution_priority_usd").diff().fill_null(
        strategy="zero").alias("execution_priority_usd"),
    pl.col("cumulative_execution_base_eth").diff().fill_null(
        strategy="zero").alias("execution_base_eth"),
    pl.col("cumulative_execution_priority_eth").diff().fill_null(
        strategy="zero").alias("execution_priority_eth"),
    pl.col("cumulative_consensus_usd").diff().fill_null(
        strategy="zero").alias("consensus_usd"),
    pl.col("cumulative_consensus_eth").diff().fill_null(
        strategy="zero").alias("consensus_eth")
])

In [5]:
blob_market_fees_df.head(5)

hr,sum_blob_cost,sum_blob_usd,cumulative_execution_base_usd,cumulative_execution_priority_usd,cumulative_execution_base_eth,cumulative_execution_priority_eth,cumulative_consensus_usd,cumulative_consensus_eth,day
str,f64,f64,f64,f64,f64,f64,f64,f64,date
"""2024-03-13 00:00:00.000 UTC""",1.9962e-10,7.9806e-7,53024.99562,977.599411,13.263473,0.244914,7.9806e-7,1.9962e-10,2024-03-13
"""2024-03-14 00:00:00.000 UTC""",1.4580e-9,0.000006,283677.668313,4615.773908,72.273649,1.170525,0.000007,1.6577e-9,2024-03-14
"""2024-03-15 00:00:00.000 UTC""",9.5329e-10,0.000004,362689.990429,7818.146453,93.47014,2.031123,0.00001,2.6110e-9,2024-03-15
"""2024-03-16 00:00:00.000 UTC""",8.7962e-10,0.000003,412353.628777,10806.630026,106.95154,2.841412,0.000013,3.4906e-9,2024-03-16
"""2024-03-17 00:00:00.000 UTC""",8.3322e-10,0.000003,452816.081158,13604.72795,118.252727,3.623104,0.000016,4.3238e-9,2024-03-17


In [6]:
# Blob Market Fees Paid
blob_market_fees_melted_df = (
    blob_market_fees_df
    .select('day', 'cumulative_consensus_usd', 'cumulative_execution_base_usd', 'cumulative_execution_priority_usd').rename({
        'day': 'time',
        'cumulative_consensus_usd': 'consensus fees (usd)',
        'cumulative_execution_base_usd': 'execution_base fees (usd)',
        'cumulative_execution_priority_usd': 'execution_priority fees (usd)'
    })
    .unpivot(index=['time'], value_name='usd_amount', variable_name='cumulative fees (usd)')
    )

In [7]:
# Define the two area charts
area_chart = alt.Chart(blob_market_fees_melted_df).mark_area().encode(
    x='time:T',
    y='usd_amount:Q',
    color='cumulative fees (usd):N'
).properties(
    title='Cumulative L2 Fees Generated From Blob Market',
    width=600,
    height=400
)

area_chart_stacked = alt.Chart(blob_market_fees_melted_df).mark_area().encode(
    x='time:T',
    y=alt.Y("usd_amount:Q").stack("normalize"),
    color='cumulative fees (usd):N'
).properties(
    title='Cumulative L2 Fees Generated From Blob Market (Normalized)',
    width=600,
    height=400
)

In [8]:
# Concatenate charts horizontally
combined_chart = alt.hconcat(area_chart, area_chart_stacked)

combined_chart

alt.HConcatChart(...)

In [163]:
cumulative_fees_pie_chart = blob_market_fees_melted_df.sort(by='time').tail(3).with_columns((pl.col('usd_amount') / 1e6).round(1))

In [164]:
cumulative_fees_pie_chart

time,cumulative fees (usd),usd_amount
date,str,f64
2024-10-27,"""consensus fees (usd)""",3.6
2024-10-27,"""execution_base fees (usd)""",6.0
2024-10-27,"""execution_priority fees (usd)""",1.3


In [165]:
# Base chart configuration with adjusted text and title
base = alt.Chart(cumulative_fees_pie_chart).encode(
    alt.Theta("usd_amount:Q").stack(True),
    alt.Color("cumulative fees (usd):N").legend()
)

# Increase the outer radius for larger pie chart
pie = base.mark_arc(outerRadius=170)

# Add black text for the numbers with increased distance from the center
l2_margin = base.mark_text(radius=190, size=12, color="black").encode(
    text="usd_amount:N"
)

# Combine the pie and text and add a title
chart = (pie + l2_margin).properties(
    title="Blob Market Fees (in Millions USD)",
    width=350,
    height=350
)

In [166]:
chart

alt.LayerChart(...)

### L2 Revenue from Blob Market

In [10]:
# https://dune.com/queries/3542365/5960052
l2_revenue_blobs_df = spice.query(3542365).with_columns(
    pl.col("dt")
    .str.strptime(pl.Date, "%Y-%m-%d %H:%M:%S%.f %Z", strict=False)
    .dt.truncate("1d").alias('day')
)

loading result from cache


In [11]:
l2_revenue_blobs_df.head(5)

dt,avg_tx_zksync,L1_cost_zksync,rev_zksync,margin_p_zksync,margin_usd_zksync,txs_zksync,daa_zksync,rev_p_daa_zks,profit_p_tx_zks,avg_tx_optimism,L1_cost_optimism,rev_optimism,margin_p_optimism,margin_usd_optimism,txs_optimism,daa_optimism,rev_p_daa_optimism,profit_p_tx_optimism,avg_tx_base,L1_cost_base,rev_base,margin_p_base,margin_usd_base,txs_base,daa_base,rev_p_daa_base,profit_p_tx_base,avg_tx_arb,L1_cost_arb,rev_arb,margin_p_arb,margin_usd_arb,txs_arb,daa_arb,rev_p_daa_arb,profit_p_tx_arb,avg_tx_blast,L1_cost_blast,rev_blast,margin_p_blast,margin_usd_blast,txs_blast,daa_blast,rev_p_daa_blast,profit_p_tx_blast,avg_tx_scroll,L1_cost_scroll,rev_scroll,margin_p_scroll,margin_usd_scroll,txs_scroll,daa_scroll,rev_p_daa_scroll,profit_p_tx_scroll,avg_tx_linea,L1_cost_linea,rev_linea,margin_p_linea,margin_usd_linea,txs_linea,daa_linea,rev_p_daa_linea,profit_p_tx_linea,day
str,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,str,i64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,str,i64,f64,f64,f64,f64,f64,f64,f64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,date
"""2023-12-01 00:00:00.000 UTC""",0.381665,134648.404897,165976.584847,0.188751,31328.17995,434875.0,147068,1.12857,0.07204,0.346189,82621.197106,100412.036948,0.177178,17790.839842,"""290050""",64895,1.5473,0.061337,0.26507,55959.441786,62587.46316,0.1059,6628.021373,"""236117""","""46611""",1.342762,0.028071,0.241226,157128.616087,199056.129175,0.210632,41927.513088,825186.0,"""112481""",1.769687,0.05081,null,null,null,null,null,null,null,null,null,0.748686,34795.980757,40737.519596,0.145849,5941.538839,"""54412""",18785,2.16862,0.109195,0.841446,105390.984379,264553.901601,0.601628,159162.917222,314404.0,59649,4.435177,0.506237,2023-12-01
"""2023-12-02 00:00:00.000 UTC""",0.342953,181043.222835,222388.17998,0.185913,41344.957145,648451.0,209445,1.061798,0.06376,0.302345,70575.654713,87700.779229,0.195268,17125.124516,"""290069""",62700,1.398737,0.059038,0.207065,57528.052904,62574.362528,0.080645,5046.309624,"""302197""","""79773""",0.784405,0.016699,0.200672,133466.922371,177988.754992,0.250138,44521.832621,886963.0,"""142827""",1.246184,0.050196,null,null,null,null,null,null,null,null,null,0.639832,41756.94754,51221.14913,0.184771,9464.20159,"""80054""",26879,1.90562,0.118223,0.786301,172222.443536,509708.855409,0.662116,337486.411873,648236.0,105105,4.849521,0.520623,2023-12-02
"""2023-12-03 00:00:00.000 UTC""",0.340858,258031.798275,301674.215399,0.144667,43642.417124,885044.0,303589,0.993693,0.049311,0.291826,63314.013893,81940.603434,0.227318,18626.589541,"""280786""",61418,1.334146,0.066337,0.206423,54507.013481,58478.123094,0.067908,3971.109612,"""283293""","""67267""",0.869343,0.014018,0.197361,128792.204765,171851.864084,0.250563,43059.659319,870750.0,"""134075""",1.281759,0.049451,null,null,null,null,null,null,null,null,null,0.630625,49565.71622,61748.290373,0.197294,12182.574153,"""97916""",31780,1.942992,0.124419,0.775241,226795.305591,665402.629888,0.659161,438607.324297,858317.0,132611,5.017703,0.511009,2023-12-03
"""2023-12-04 00:00:00.000 UTC""",0.296548,185851.397083,224133.780434,0.170801,38282.383351,755809.0,170193,1.316939,0.050651,0.762593,259768.047241,285887.112197,0.091361,26119.064955,"""374888""",79969,3.574974,0.069672,0.325193,74363.766682,79131.042987,0.060245,4767.276304,"""243336""","""53547""",1.477787,0.019591,0.308763,246850.95694,314591.42664,0.215328,67740.4697,1.018878e6,"""118402""",2.656977,0.066485,null,null,null,null,null,null,null,null,null,0.929258,53020.38623,61225.097725,0.134009,8204.711495,"""65886""",20682,2.960308,0.124529,1.128759,32827.161583,532341.92209,0.938334,499514.760508,471617.0,77810,6.841562,1.059153,2023-12-04
"""2023-12-05 00:00:00.000 UTC""",0.311775,333764.553542,350324.730576,0.047271,16560.177034,1.123646e6,557067,0.628874,0.014738,0.732047,246464.036346,244641.285839,-0.007451,-1822.750507,"""334188""",66064,3.703095,-0.005454,0.417378,

### L2 Margins pre/post Dencun

In [12]:
l2_margin_df = l2_revenue_blobs_df.select([pl.col(col) for col in l2_revenue_blobs_df.columns if "margin_p" in col or col == "day"])
l2_margin_usd_df = l2_revenue_blobs_df.select([pl.col(col) for col in l2_revenue_blobs_df.columns if "margin_usd" in col or col == "day"])

In [13]:
weekly_l2_margin = l2_margin_df.with_columns(pl.col('day').dt.round('1w').alias('week')).group_by('week').agg(
    pl.col('margin_p_optimism').mean().alias('optimism'),
    pl.col('margin_p_base').mean().alias('base'),
    pl.col('margin_p_arb').mean().alias('arb'),
    pl.col('margin_p_blast').mean().alias('blast'),
    pl.col('margin_p_scroll').mean().alias('scroll'),
    pl.col('margin_p_linea').mean().alias('linea')
).sort(by='week')

weekly_l2_margin_usd = l2_margin_usd_df.with_columns(pl.col('day').dt.round('1w').alias('week')).group_by('week').agg(
    pl.col('margin_usd_optimism').mean().alias('optimism'),
    pl.col('margin_usd_base').mean().alias('base'),
    pl.col('margin_usd_arb').mean().alias('arb'),
    pl.col('margin_usd_blast').mean().alias('blast'),
    pl.col('margin_usd_scroll').mean().alias('scroll'),
    pl.col('margin_usd_linea').mean().alias('linea')
).sort(by='week')

In [14]:
melted_weekly_l2_margin_df = weekly_l2_margin.unpivot(
    index=["week"], 
    on=["optimism", "base", "arb", "blast", "scroll", "linea"],
    variable_name="l2_margin", 
    value_name="margin_value"
)

melted_weekly_l2_margin_usd_df = weekly_l2_margin_usd.unpivot(
    index=["week"], 
    on=["optimism", "base", "arb", "blast", "scroll", "linea"],
    variable_name="l2_margin", 
    value_name="margin_value"
)

In [15]:
chart = alt.Chart(melted_weekly_l2_margin_df).mark_line().encode(
    x=alt.X('week:T', title='Week'),
    y=alt.Y('margin_value:Q', title='Margin Value (%)', 
            axis=alt.Axis(format='.0%', tickCount=5)),
    color='l2_margin:N'
)

rule = alt.Chart(pl.DataFrame({'week': ['2024-03-13']})).mark_rule(
    color='black',
    strokeDash=[4, 4],
    size=3
).encode(
    x='week:T'
)

text = alt.Chart(pl.DataFrame({
    'week': ['2024-03-13'],
    'label': ['Dencun Start']
})).mark_text(
    align='left',
    dx=-75,
    dy=-50,
    color='black'
).encode(
    x='week:T',
    text='label:N'
)


final_chart = (
    (chart + rule + text)
    .properties(
        title="L2 Margin pre/post Dencun (weekly)",
        height=400,
        width=600
    )
)

final_chart

alt.LayerChart(...)

### L2 Revenue from Blob Market

In [24]:
l2_cumulative_margin_usd = melted_weekly_l2_margin_usd_df.with_columns(
    (pl.col('margin_value').cum_sum().over('l2_margin')).alias('cumulative_l2_margin_usd')
)

In [100]:
l2_cumulative_margin_usd

week,l2_margin,margin_value,cumulative_l2_margin_usd
date,str,f64,f64
2023-12-04,"""optimism""",17064.513187,17064.513187
2023-12-11,"""optimism""",29267.026109,46331.539296
2023-12-18,"""optimism""",3650.411783,49981.951079
2023-12-25,"""optimism""",26358.587129,76340.538209
2024-01-01,"""optimism""",38062.872189,114403.410397
…,…,…,…
2024-09-30,"""linea""",10722.867498,4.1298e6
2024-10-07,"""linea""",11231.298182,4.1410e6
2024-10-14,"""linea""",12508.992522,4.1535e6


In [102]:
l2_cumulative_revenue_total = l2_cumulative_margin_usd.sort(by='week').tail(6).with_columns((pl.col('cumulative_l2_margin_usd') / 1e6).round(2))

In [103]:
l2_cumulative_revenue_total

week,l2_margin,margin_value,cumulative_l2_margin_usd
date,str,f64,f64
2024-10-28,"""optimism""",10999.531361,1.55
2024-10-28,"""base""",148018.168099,7.64
2024-10-28,"""arb""",19030.702671,2.91
2024-10-28,"""blast""",4666.309904,1.52
2024-10-28,"""scroll""",5847.260204,2.04
2024-10-28,"""linea""",26671.301902,4.2


In [171]:
# Base chart configuration with adjusted text and title
base = alt.Chart(l2_cumulative_revenue_total).encode(
    alt.Theta("cumulative_l2_margin_usd:Q").stack(True),
    alt.Color("l2_margin:N").legend()
)

# Increase the outer radius for larger pie chart
pie = base.mark_arc(outerRadius=170)

# Add black text for the numbers with increased distance from the center
l2_margin = base.mark_text(radius=190, size=12, color="black").encode(
    text="cumulative_l2_margin_usd:N"
)

# Combine the pie and text and add a title
chart = (pie + l2_margin).properties(
    title="L2 Cumulative Margins (in Millions USD)",
    height=350,
    width=350
)

In [172]:
chart

alt.LayerChart(...)

In [72]:
l2_margin_usd_stacked_area_chart = alt.Chart(l2_cumulative_margin_usd).mark_area().encode(
    x="week:T",
    y=alt.Y("cumulative_l2_margin_usd:Q").stack("normalize"),
    color="l2_margin:N"
).properties(
        title="L2 Margins (usd)",
        height=400,
        width=600
    )

l2_margin_usd_area_chart = alt.Chart(l2_cumulative_margin_usd).mark_area().encode(
    x="week:T",
    y=alt.Y("cumulative_l2_margin_usd:Q"),
    color="l2_margin:N"
).properties(
        title="L2 Margins (usd)",
        height=400,
        width=600
    )

In [46]:
# Concatenate charts horizontally
l2_margin_charts = alt.hconcat(l2_margin_usd_area_chart, l2_margin_usd_stacked_area_chart)

l2_margin_charts

alt.HConcatChart(...)